# [문제] Managing Conversation History
-trim_messages()
-RunnablePassthrough
-itemgetter()

세션 ID 설정하고, 대화를 진행합니다.(multi-turn conversation)  
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
## 1. 모듈 import
from dotenv import load_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


## 7. trim 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

## 8. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm 
)

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}}      
    )

    print('\n' + '=' * 50 + '\n')

옥찌  
이메일: your-email@example.com | 전화번호: 010-1234-5678  

---

## 경력 요약

저는 창의적이고 문제 해결 능력이 뛰어난 프리랜서 프로젝트 매니저입니다. 다양한 분야의 프로젝트를 성공적으로 관리한 경험을 통해 뛰어난 커뮤니케이션 및 조직 관리 능력을 갖추고 있습니다. 지속 가능한 성과와 높은 팀 성과를 이끌어내는 데 관심을 가지고 있습니다.

---

## 경력

**프리랜서 프로젝트 매니저**  
2020년 3월 - 현재

- 다양한 클라이언트를 대상으로 프로젝트 계획 및 실행 관리
- 팀원들과의 긴밀한 협력을 통해 목표를 초과 달성
- 프로젝트 진행 상황 모니터링 및 문제 해결 프로세스 개발

**XYZ 회사, 프로젝트 코디네이터**  
2017년 6월 - 2020년 2월

- 팀 내 다양한 부서와 협력하여 프로젝트 기한 및 목표 달성
- 프로젝트 상태 보고서를 작성하고 이해관계자에게 전달
- 효율적인 프로젝트 관리 툴을 사용하여 작업 흐름 최적화

---

## 학력

**ABC 대학교, 문학사**  
주요 과목: 커뮤니케이션, 경영학  
2013년 3월 - 2017년 2월

---

## 기술

- 프로젝트 관리: Agile, Scrum, Waterfall
- 도구/소프트웨어: MS Office, Trello, Asana, JIRA
- 언어: 한국어(모국어), 영어(중급), 일본어(초급)

---

## 자격증 및 수상

- PMP(Project Management Professional) 자격증, 2019년
- ABC 대학교 학업 우수상, 2016년

---

## 기타 활동

- 지역 사회의 자원봉사 프로그램 기획 및 참여
- 다문화 환경에서의 협동과 리더십 연구 참여

---

## 개인 관심사

- 지속 가능한 에너지 프로젝트
- 새로운 기술 트렌드 연구
- 문화 및 예술 전시회 탐방

---

이력서에 관심을 가져주셔서 감사합니다. 추가 정보가 필요하시면 언제든지 연락 주시기 바랍니다.  

(

In [2]:
get_history('1234')

InMemoryChatMessageHistory(messages=[HumanMessage(content='이력서 작성을 어떻게 해야 하나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='이력서 작성 방법에 대해 안내해 드리겠습니다. 효과적인 이력서는 자신의 경력과 역량을 명확하고 간결하게 전달하여 채용 담당자의 관심을 끄는 것이 중요합니다. 아래 단계별로 작성 방법을 설명드리겠습니다.\n\n1. 기본 정보\n- 이름, 연락처(전화번호, 이메일), 주소(선택사항)를 명확히 기재하세요.\n- 최근 연락이 가능한 연락처를 기입하는 것이 좋습니다.\n\n2. 자기소개 또는 경력개요\n- 간단하게 본인의 전문성, 강점, 지원 동기를 요약하세요.\n- 해당 직무에 적합한 역량을 강조하는 것이 좋습니다.\n\n3.', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-335e9686-c8de-41d1-82f8-16a4b91223d6-0'), HumanMessage(content='이력서 작성해주세요!', additional_kwargs={}, response_metadata={}), AIMessage(content='물론입니다! 지원자의 기본 정보, 경력 사항, 학력, 기술 및 자격증, 그리고 자기소개 등을 포함한 예시 이력서를 아래에 작성해 드리겠습니다. 참고용으로 활용하시거나 필요에 맞게 수정하시면 됩니다.\n\n---\n\n**이력서**\n\n**개인정보**  \n이름: 홍길동  \n생년월일: 1990년 5월 15일  \n연락처: 010-1234-5678  \n이메일: honggildong@email.com  \n주소: 서울시 강남구 테헤란로 123  \n\n---\n

In [3]:
for info in get_history('1234').messages:
    print(info.content)
    print('*' * 50)

이력서 작성을 어떻게 해야 하나요?
**************************************************
이력서 작성 방법에 대해 안내해 드리겠습니다. 효과적인 이력서는 자신의 경력과 역량을 명확하고 간결하게 전달하여 채용 담당자의 관심을 끄는 것이 중요합니다. 아래 단계별로 작성 방법을 설명드리겠습니다.

1. 기본 정보
- 이름, 연락처(전화번호, 이메일), 주소(선택사항)를 명확히 기재하세요.
- 최근 연락이 가능한 연락처를 기입하는 것이 좋습니다.

2. 자기소개 또는 경력개요
- 간단하게 본인의 전문성, 강점, 지원 동기를 요약하세요.
- 해당 직무에 적합한 역량을 강조하는 것이 좋습니다.

3.
**************************************************
이력서 작성해주세요!
**************************************************
물론입니다! 지원자의 기본 정보, 경력 사항, 학력, 기술 및 자격증, 그리고 자기소개 등을 포함한 예시 이력서를 아래에 작성해 드리겠습니다. 참고용으로 활용하시거나 필요에 맞게 수정하시면 됩니다.

---

**이력서**

**개인정보**  
이름: 홍길동  
생년월일: 1990년 5월 15일  
연락처: 010-1234-5678  
이메일: honggildong@email.com  
주소: 서울시 강남구 테헤란로 123  

---

**자기소개 및 경력개요**  
신뢰성과 성실성을 바탕으로 문제
**************************************************
이름: 박보검, 지원분야: 웹 개발 신입, 보유 능력: Python, HTML, CSS, LLM
**************************************************
물론입니다! 아래는 지원자 박보검 님의 "웹 개발 신입" 분야에 적합한 이력서 예시입니다. 자연스럽고 전문적인 느낌을 살려서 작성하였으며, 필요에 따라 자유롭게 수정